In [ ]:
import requests
from datetime import datetime, timedelta

# Toggl API settings
API_TOKEN = 'YOUR_TOGGL_API_TOKEN'
HEADERS = {
    'Authorization': f'Basic {API_TOKEN}',
    'Content-Type': 'application/json'
}
BASE_URL = 'https://api.track.toggl.com/api/v8/'

# Calculate date range for the last month
end_date = datetime.now().date()
start_date = end_date - timedelta(days=30)

# Retrieve time entries
time_entries_response = requests.get(
    f'{BASE_URL}time_entries?start_date={start_date}&end_date={end_date}',
    headers=HEADERS
)
time_entries = time_entries_response.json()

# Retrieve projects
projects_response = requests.get(f'{BASE_URL}projects', headers=HEADERS)
projects = projects_response.json()

In [ ]:
import pandas as pd

df_time_entries = pd.DataFrame(time_entries)
df_projects = pd.DataFrame(projects)

In [ ]:
# Join time entries with projects
df_joined = df_time_entries.merge(df_projects, left_on='pid', right_on='id', how='inner')

# Group by project and sum durations
df_duration = df_joined.groupby('name')['duration'].sum().sort_values(ascending=False).reset_index()

In [ ]:
df_unique_entries = df_joined.groupby('name').size().sort_values(
    ascending=False).reset_index(name='unique_entries_count')